# Klassifikation: Vorhersage von Weinqualitäten

Quelle:
https://archive.ics.uci.edu/ml/datasets/wine+quality  

Der Datensatz enhält Daten über die  chemischen Zusammensetzung von Rotweinen und ihrer Qualität. Er wurde veröffentlicht von Paulo Cortez, Antonio Cerdeira, Fernando Almeida, Telmo Matos and Jose Reis. (ISSN: 0167-9236)

Der Datensatz wurde für die Verwendung in diesem Notebook leicht modifiziert.

<hr style="border:1px solid gray"> </hr>

## Inhalt


1. [Geschäftverständnis / Aufgabe](#kap1)


2. [Datenverständnis](#kap2) 


3. [Datenvorbereitung](#kap3) 

    3.1 [Aufteilung in Trainings- und Testdatensatz](#kap31)  
    3.2 [Standardisierung der Daten](#kap32)  
    3.3 [Visualisierung](#kap33)  
    3.4 [Merkmalsauswahl anhand der Korrelationsmatrix](#kap34) 


4. [Modellierung](#kap4) 

    4.1 [Entscheidungsbaum](#kap41)  
    4.2 [kNN (k-Nächste Nachbarn)](#kap42)  
    4.3 [SVM (Support Vector Machine)](#kap43)    
    

5. [Fazit](#kap5)

<hr style="border:1px solid gray"> </hr>

## 1. Geschäftsverständnis / Aufgabe <a name="kap1"></a>

Es liegt ein Datensatz vor, der Daten zur chemischen Zusammensetzung von Rotweinen enthält und eine Aussage zur jeweiligen Qualität der Weine trifft. Es soll untersucht werden, ob die Weinqualität anhand der chemischen Daten bestimmt werden kann.

## 2. Datenverständnis <a name="kap2"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
csv_path = "M4_Video_Weinqualitaet.csv"
df = pd.read_csv(csv_path, sep=',', index_col=0)  
df

Zum Datensatz: 

Der Datensatz enthält 1599 Zeilen, dies sind die chemischen Analysewerte von verschiedenen Rotweinproben, die ausgewertet wurden. Es wurden jeweils 12 Eigenschaften (= Spalten) erfasst. Die Namen der Spalten bedeuten folgendes:

- `fixed acidity` = Festsäuregehalt 	
- `volatile acidity` = Flüchtige Säuren
- `citric acid` = Zitronensäuregehalt
- `residual sugar` = Restzuckergehalt 
- `chlorides` = Chloride 	
- `free sulfur dioxide` = Freier Schwefeldioxidgehalt
- `total sulfur dioxide` Gesamt-Schwefeldioxidgehalt
- `density` = Dichte
- `pH` = pH-Wert
- `sulphates` = Sulfatgehalt 	
- `alcohol` = Alkoholgehalt
- `quality` = Qualität

Die *Qualität* wurde auf Grundlage sensorischer Daten erhoben und soll das Zielmerkmal dieser Modellbildung sein.

Für die Betrachtung dieses Problems ist es nicht nötig, chemisches Hintergrundwissen zu besitzen.  
Grundsätzlich kann es aber zum Beispiel für die Featureauswahl, die Festlegung der Aufgabenstellung oder für die Implementierung des Modells nützlich sein, über das Hintergrundwissen des jeweiligen Datensatzes zu verfügen. Data Mining Probleme sind daher häufig interdisziplinäre Aufgabenfelder, in denen verschiedene Wissensgebiete zusammenfließen.

In [ ]:
df.info()

Der Datensatz enthält 1599 Einträge. Für jedes Merkmal sind alle Felder vollständig gefüllt (Zur Erinnerung: Dies zeigt die Spalte `Non-Null Count` = 'Nicht-Null Anzahl' an. 
In der Spalte `Dtype` kann der Datentyp abgelesen werden. Alle Merkmale liegen als float vor, das Zielmerkmal `quality` ist als integer eingetragen).

In [ ]:
df.describe()

Alle enthaltenen Daten sind numerisch, sodass keine weiteren Anpassungen vorgenommen werden müssen. Die Merkmale haben jedoch unterschiedliche Größenordnungen, sodass eine Standardisierung der Daten vorgenommen werden sollte (s. [Kapitel 3.2](#kap32)).

In [ ]:
df.hist(figsize=(12,12));

Die Histogramme zeigen, dass es keine Ausreißer gibt und die Merkmale sehr unterschiedlich verteilt sind.  
Beim Merkmal `quality` fällt auf, dass die Verteilung der Einträge sehr ungleich ist: 1382 Einträge gehören der Qualität 0 (gut) an, während für die Weinqualität 1 (schlecht) nur 217 Daten vorliegen.  

In [ ]:
df['quality'].value_counts()

## 3. Datenvorbereitung <a name="kap3"></a>

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

In diesem Kapitel soll der Wert vor allem auf den Verfahren liegen. Dennoch muss der Datensatz für die Modellierung vorbereitet werden. Führen Sie die einzelnen Schritte der Datenvorbereitung ([Kapitel 3](#kap3)) aus und lesen Sie die zugehörige Kommentierung. 
</div>

### 3.1 Aufteilung in Trainings- und Testdatensatz <a name="kap31"></a>

Es wird wieder begonnen, den Datensatz mit `train_test_split` in einen Trainings- und einen Testdatensatz aufzuteilen (Erinnerung: Der Testdatensatz wird genutzt, um die Qualität des Modells zu überprüfen. Dazu werden die Merkmale des Testdatensatzes genutzt, um eine Vorhersage für das Zielmerkmal zu erstellen. Diese kann dann mit dem bekannten Zielmerkmal des Testdatensatzes abgeglichen werden).

Dieser Schritt sollte vor der Modellierung und auch vor der weiteren Bearbeitung der Daten und der Featureauswahl erfolgen. Dadurch können die Testdaten so betrachtet werden, als würden diese gar nicht vorliegen. Das Modell und die zugehörigen Vorüberlegungen können so rein auf Grundlage der Trainingsdaten erstellt werden.

In [ ]:
from sklearn.model_selection import train_test_split
train_set,test_set = train_test_split(df, random_state=0, test_size=0.2, stratify = df['quality'])

### 3.2 Standardisierung der Daten <a name="kap32"></a>

Wie bereits oben angekündigt, müssen die Daten vor der Modellierung umskaliert werden, da die Werte der Merkmale verschiedene Größenordnungen besitzen (z.B. im Schnitt 0.27 bei `citric acid` verglichen mit 46 bei`total sulfur dioxide`). Hier soll nun der `StandardScaler` (vergleiche Modul 3) verwendet werden. 

In [ ]:
from sklearn.preprocessing import StandardScaler

# Nur die Merkmale werden standardisiert, daher wird eine Liste erstellt, die nur die Merkmale ohne das Zielmerkmal enthält
features = list(df.drop(['quality'], axis=1).columns)

# Skalierung wird an train_set angepasst
scaler = StandardScaler()
scaler.fit(train_set[features])

# Skalierung der Trainingsdaten
train_features_scaled = pd.DataFrame(scaler.transform(train_set[features]), columns=features, index=train_set.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
train_set_scaled = pd.concat([train_features_scaled, train_set['quality']], axis=1)

# Skalierung der Testdaten
test_features_scaled = pd.DataFrame(scaler.transform(test_set[features]), columns=features, index=test_set.index)
# Zusammenfügen der skalierten Merkmale und das Zielmerkmal (nicht skaliert) zu einem Dataframe
test_set_scaled = pd.concat([test_features_scaled, test_set['quality']], axis=1)

train_set_scaled.head()

Es ist zu beachten, dass nun die ursprünglichen Maßeinheiten keinen Sinn mehr ergeben und eine Transformation durchgeführt wurde, die chemisch gesehen keinen Sinn hat. Hier werden die Daten jedoch nicht aus der chemischen Perspektive, sondern als Data Science Projekt betrachtet, sodass hier alles erlaubt ist, was für die Modellierung die besten Ergebnisse liefert.

Ein Blick auf die Statistiken der Daten zeigt, dass nun tatsächlich die Mittelwerte (mean) der transformierten Merkmale ungefähr 0 und die Standardabweichungen (std) ungefähr 1 sind:

In [ ]:
train_set_scaled.describe()

### 3.3. Visualisierung <a name="kap33"></a>

In [ ]:
fig=plt.figure(figsize=(15,10))
columns = 4
rows = 3
for i in range(1, 12):
    fig.add_subplot(rows, columns, i)
    sns.barplot(x = 'quality', y = df.iloc[:,i-1], data=train_set_scaled) 
plt.tight_layout()

Die obige Darstellung lässt bereits erahnen, dass sich die Werte einiger Merkmale zwischen den Qualitäten nicht unterscheiden. Die ist beispielsweise beim pH-Wert (`pH`) und bei der Dichte (`density`) der Fall.

### 3.4 Merkmalsauswahl anhand der Korrelationsmatrix <a name="kap34"></a>

Der Eindruck aus der vorhergehenden Grafik lässt sich durch die Korrelationsmatrix noch vertiefen: 

In [ ]:
plt.figure(figsize=(10, 7))
sns.heatmap(train_set_scaled.corr(),annot=True, vmin=-1, vmax=1);

Es wird zunächst die Korrelation der Merkmale mit dem Ziel `quality` betrachtet:  
- `alcohol` hat mit 0.44 die höchste Korrelation, gefolgt von
- `volatile acidity` mit -0.28
- `citric acid` mit 0.24
- `sulphates` mit 0.2

Bei der Betrachtung der Korrelation der Merkmale zueinander fällt auf, dass die Merkmale `volatile acidity` und `citric acid` miteinander korrelieren (-0.54). Aus diesem Grund wird `citric acid` nicht als Merkmal für die Modellierung genutzt.

Die Grundlage für die Modellierung bilden somit die Merkmale:
- `alcohol`
- `volatile acidity` 
- `sulphates`

# 4. Modellierung <a name="kap4"></a>

Zunächst werden Trainings- und Testdaten auf die gewünschten Merkmale aus [Kapitel 3.4](#kap34) eingeschränkt: 

In [ ]:
# Trainingsdaten für die Modellierung mit ausgewählten Merkmalen
X_train = train_set_scaled[['volatile acidity', 'sulphates', 'alcohol']]
y_train = train_set_scaled[['quality']].values.ravel()

# Testdaten für die Modellierung mit ausgewählten Merkmalen
X_test = test_set_scaled[['volatile acidity', 'sulphates', 'alcohol']]
y_test = test_set_scaled[['quality']].values.ravel()

### 4.1 Entscheidungsbaum <a name="kap41"></a>

Entscheidungsbäume (Decision Trees) wurden bereits im End-to-end Projekt vorgestellt. Es handelt sich dabei um gerichtete Entscheidungsdiagramme.  
Hierbei steht jede Ebene für eine Entscheidungsregel, deren Verzweigung zu weiteren Entscheidungsebenen führt. Dabei wird an jedem Knoten entschieden, ob ein Eintrag dem linken oder rechten Ast zugeordnet wird.
Die letzte Ebene bildet die Ergebnissebene, die die Klasseneinteilung bestimmt.

Entscheidungsbäume haben den großen Vorteil, dass ihre Ergebnisse "interpretierbar" sind, d.h. die Entscheidung, warum ein gewisser Eintrag im Datensatz einer gewissen Klasse zugeordnet wird, ist leicht nachvollziehbar. Diese Erklärbarkeit macht Entscheidungsbäume zu einem beliebten Machine Learning Verfahren, das häufig in Anwendungsgebieten wie der Medizintechnik, Pharmaindustrie und der Bankenbranche zum Einsatz kommt.

Mathematisch werden die Entscheidungsregeln gebildet, indem mithilfe der Entropiefunktion oder dem Gini-Index bestimmt wird, wie viel Information in den Merkmalen enthalten ist und dann verglichen wird, welches Merkmal den größten "Informationszuwachs" liefert.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf_dt = DecisionTreeClassifier(max_depth = 2)
clf_dt.fit(X_train,y_train)

In [ ]:
from sklearn import tree
_, ax = plt.subplots(figsize=(10,6))
tree.plot_tree(clf_dt, feature_names=X_train.columns, class_names=['gut', 'schlecht'], filled=True);

Nachfolgend werden die Gütemaße aus dem ersten Video des vierten Moduls bestimmt. 

In [ ]:
print("Korrektklassifikationsrate auf Trainingsdaten:", clf_dt.score(X_train, y_train))
print("Korrektklassifikationsrate auf Testdaten:",clf_dt.score(X_test,y_test))

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score, precision_score

y_pred_dt = clf_dt.predict(X_test)
print(confusion_matrix(y_test, y_pred_dt))
print('Recall:', recall_score(y_test, y_pred_dt, zero_division=False))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=False))

### 4.2 kNN (k-Nächste Nachbarn) <a name="kap42"></a>

Der kNN-Algorithmus ermittelt für ein neu zu klassifizierendes Beispiel die Nachbarn, also die (geometrisch betrachtet) umliegenden Beispiele. Die am häufigsten in der 'Nachbarschaft' vorkommende Klasse bestimmt dann die Klassenzuordnung. 

Die Standard Syntax für scikit-learn Verfahren funktioniert auch hier. Zunächst wird das k-Nächste Nachbarn Modell für k=1 angewendet:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf_nn = KNeighborsClassifier(1) # Initialisierung des Modells
clf_nn.fit(X_train, y_train) # Trainieren des Modells anhand der Trainingsdaten

Das so erstellte Modell kann nun auch auf die Testdaten angewandt werden.  
Außerdem kann die Korrektklassifikationsrate des Modells auf die Trainings- und Testdaten, sowie die Konfusionsmatrix ausgegeben werden.

In [ ]:
y_pred_nn = clf_nn.predict(X_test)
print("Korrektklassifikationsrate auf Trainingsdaten:", clf_nn.score(X_train,y_train))
print("Korrektklassifikationsrate auf Testdaten:",clf_nn.score(X_test, y_test))
print(confusion_matrix(y_test, y_pred_nn))

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

Erforschen Sie die nachfolgend systematische Veränderung des Wertes k. 

</div>

Die Zahl der k-Nächsten Nachbarn wirkt sich auf das Modell aus. Daher wird im Folgenden ermittelt, welches k die "besten" Ergebnisse liefert.

In [ ]:
# Erstellen von zwei leeren Listen, in die die Korrektklassifikationsrate für die verschiedenen k eingetragen werden
score_test = []
score_train = []

for k in range(1,10):
    clf = KNeighborsClassifier(n_neighbors = k)
    clf.fit(X_train, y_train)
    s1 = clf.score(X_train, y_train)
    s2 = clf.score(X_test, y_test)
    score_train.append(s1)
    score_test.append(s2)

plt.plot(range(1,10),score_train, '-o', label="Trainingsdaten");
plt.plot(range(1,10),score_test, '-o', label = "Testdaten");
plt.title("Korrektklassifikationsrate bei verschiedenen k's");
plt.xlabel("k");
plt.ylabel("Korrektklassifikationsrate");
plt.legend();

Für das kNN-Modell wird k = 6 gewählt:

In [ ]:
clf_nn_opt = KNeighborsClassifier(n_neighbors=6)
clf_nn_opt.fit(X_train,y_train)
print("Korrektklassifikationsrate auf Trainingsdaten:", clf_nn_opt.score(X_train, y_train))
print("Korrektklassifikationsrate auf Testdaten:", clf_nn_opt.score(X_test, y_test))
y_pred_nn_opt = clf_nn_opt.predict(X_test)
print(confusion_matrix(y_test, y_pred_nn_opt))
print('Precision:', precision_score(y_test, y_pred_nn_opt))
print('Recall:', recall_score(y_test, y_pred_nn_opt))

### 4.3 SVM (Support Vector Machine) <a name="kap43"></a>

Im Gegensatz zum kNN-Algorithmus werden bei SVMs nicht die benachbarten Klassen gesucht, sondern es erfolgt eine geometrische Trennung der Klassen durch eine (ursprünglich) lineare Funktion. 

Die Standard Syntax für scikit-learn Verfahren funktioniert auch hier. Zunächst wird die SVM ohne Variation der Parameter angewendet:

In [ ]:
from sklearn import svm
clf_svm = svm.SVC() # Initialisierung des Modells, hier SVC = Support Vector Classifier
clf_svm.fit(X_train, y_train) # Trainieren des Modells anhand der Trainingsdaten

Damit ergibt sich:

In [ ]:
print("Korrektklassifikationsrate auf Trainingsdaten:", clf_svm.score(X_train, y_train))
print("Korrektklassifikationsrate auf Testdaten:",clf_svm.score(X_test, y_test))

In [ ]:
y_pred_svm = clf_svm.predict(X_test)
print(confusion_matrix(y_test, y_pred_svm))

## 5. Fazit <a name="kap5"></a>

Es wurden einige Klassifikationsverfahren vorgestellt, hierbei wurde...

- festgestellt, dass mit Hilfe von sklearn die verschiedenen Modelle alle nach demselben Schema angewendet werden:
    1. Importieren des Klassifikationsverfahren
    2. Initiieren des Modells
    3. Trainieren des Modells anhand der Trainingsdaten
    4. Anwendung des trainierten Modells auf die Testdaten
    5. Ausgabe der Gütemaße


- in diesem Beispiel von allen Modellen eine hohe Korrektklassifikationsrate erreicht, wobei die Weine schlechter Qualität aber nur sehr schlecht erkannt werden. 


- festgestellt, dass eine genaue Betrachtung der Gütemaße in Bezug auf die Problemstellung nötig ist (bspw. "wann Precision - wann Recall?", bspw. "ist die Korrektklassifikationsrate genau so aussagekräftig wie die Konfusionsmatrix?"), um eine Aussage treffen zu können, ob es sich um ein gutes Modell handelt.